In [2]:
from pathlib import Path

import pandas as pd # type: ignore

import src.balance_tracker_pipeline_v2
import src.cdutils.database.sliding_window
import src.excel_output
import src.monthly_delta
from src._version import __version__

# Fetch Data from COCC
data_prior, data_current = src.cdutils.database.sliding_window.fetch_data()

_, data_prior_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)

In [3]:
def weighted_avg_rate(df: pd.DataFrame, title: str='Weighted Avg Rate') -> pd.DataFrame:
    """
    Create weighted average rate
    """
    df = df.copy()
    df['WeightedRate'] = df['Net Balance'] * df['noteintrate']
    grouped_df = df.groupby('Category').apply(
        lambda x: x['WeightedRate'].sum() / x['Net Balance'].sum(), include_groups=False
    ).reset_index(name=title).copy()
    return grouped_df

In [4]:
import numpy as np

In [5]:
def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

    Args:
        df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

    Returns:
        df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
    """
    # Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
    df = df.copy()
    
    df['noteintrate'] = np.where(
        (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
        df['noteintrate']))))))))))))))
    )

    # Create total yield
    total_yield = weighted_avg_rate(df, title='Total Loan Yield')

    # Create new loan yield
    datetime_cols = ['effdate','origdate']
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col])

    new_loan_df = df[
        (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
    ].copy()

    new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield')

    df['Unadvanced'] = df['Total Exposure'] - df['Net Balance']

    # Unadvanced
    unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
    unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

    # Merge
    merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
    merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

    return merged_df




In [6]:
merged_df = yield_and_unadvanced_creation(full_data_current)

In [7]:
merged_df

,Category,New Loan Yield,Total Loan Yield,Unadvanced Funds
0,C&I,0.093160,0.063741,1.517306e+08
1,CRE,0.069331,0.055728,1.520219e+08
2,Consumer,0.091182,0.078189,5.167380e+05
3,HOA,0.064869,0.065821,1.612136e+07
4,Indirect,0.056459,0.059672,0.000000e+00
5,Residential,0.064305,0.042032,8.261643e+07


In [1]:

import cdutils.deduplication # type: ignore
import cdutils.database.connect # type: ignore
import cdutils.input_cleansing # type: ignore
from sqlalchemy import text # type: ignore
import pandas as pd

def fetch_from_acctuserfield():
    """
    Gets data from COCC
    """
    wh_acctuserfields = text(f"""
    SELECT
        *
    FROM 
        OSIBANK.WH_ACCTUSERFIELDS a
    """)

    queries = [
        {'key':'wh_acctuserfields', 'sql':wh_acctuserfields, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [2]:
acctuser = fetch_from_acctuserfield()

In [3]:
acctuser = acctuser['wh_acctuserfields'].copy()

In [4]:
acctuser

,acctnbr,acctuserfieldcd,acctuserfieldcddesc,acctuserfieldvalue,acctuserfieldvaluedesc,acctdatelastmaint,rundate,datelastmaint
0,26105802,WITH,Withholding Status,4,"Exempt, Confirmed",2018-08-04 14:21:10,2025-04-22,2025-04-22 21:40:05
1,26105993,DDOC,Digital Document,MM_STMNT,D3 Electronic Statement,2018-08-04 14:21:10,2025-04-22,2025-04-22 21:40:05
2,26106124,ODP,ODP Status Code,1,Active,2018-08-04 14:21:10,2025-04-22,2025-04-22 21:40:05
3,26106272,ODP,ODP Status Code,1,Active,2018-08-04 14:21:10,2025-04-22,2025-04-22 21:40:05
4,26106299,REOD,ATM/POS Opt In Flag,N,Opt Out,2018-08-04 14:21:10,2025-04-22,2025-04-22 21:40:05
...,...,...,...,...,...,...,...,...
497286,150414036,SPLT,Dealer Split Rate,0.0200,None,2019-12-27 09:38:37,2025-04-22,2025-04-22 21:40:05
497287,150422237,BRNC,Origination Branch,BCSB - Indirect Lending,None,2020-01-16 13:28:37,2025-04-22,2025-04-22 21:40:05
497288,150331339,DDOC,Digital Document,LN_BILLS,D3 Electronic Loan Bill,2019-12-02 18:18:57,2025-04-22,2025-04-22 21:40:05
497289,28058372,ODP,ODP Status Code,4,Ineligible Account by Title,2019-12-16 14:42:39,2025-04-22,2025-04-22 21:40:05


In [5]:
splt = acctuser[acctuser['acctuserfieldcd'] == 'SPLT'].copy()

In [6]:
assert splt['acctnbr'].is_unique, "Failure"

In [13]:
"""
Using the lookup query to inspect the DB tables
"""

import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

def fetch_data():
    """
    Main data query
    """
    # Engine 1
    lookup_df = text("""
    SELECT 
        *
    FROM 
        sys.all_tab_columns col
    """)

    queries = [
        # {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
        {'key':'lookup_df', 'sql':lookup_df, 'engine':2},
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [14]:
lookup = fetch_data()

In [15]:
lookup = lookup['lookup_df'].copy()

In [16]:
lookup

,owner,table_name,column_name,data_type,data_type_mod,data_type_owner,data_length,data_precision,data_scale,nullable,...,char_used,v80_fmt_image,data_upgraded,histogram,default_on_null,identity_column,evaluation_edition,unusable_before,unusable_beginning,collation
0,XDB,XDB$IMPORT_TT_INFO,ID,RAW,None,None,8,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
1,XDB,XDB$IMPORT_TT_INFO,FLAGS,RAW,None,None,4,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
2,XDB,XDB$IMPORT_TT_INFO,LOCALNAME,VARCHAR2,None,None,2000,NaN,NaN,Y,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
3,XDB,XDB$IMPORT_TT_INFO,NMSPCID,RAW,None,None,8,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
4,XDB,XDB$IMPORT_TT_INFO,GUID,RAW,None,None,16,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28689,SYS,USER_XML_SCHEMA_ELEMENTS,GLOBAL,RAW,None,None,1,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
28690,SYS,KU$_ASSOC_VIEW,OBJ_TYPE,NUMBER,None,None,22,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
28691,SYS,EXU8VEWU,DEFER,NUMBER,None,None,22,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
28692,SYS,KU$_USER_VIEW,VERS_MAJOR,CHAR,None,None,1,NaN,NaN,Y,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP


In [ ]:

import cdutils.deduplication # type: ignore
import cdutils.database.connect # type: ignore
import cdutils.input_cleansing # type: ignore
from sqlalchemy import text # type: ignore
import pandas as pd
from datetime import date, datetime
# ------------------------------------------------------------------
# --- 1.  Runtime inputs -------------------------------------------
# ------------------------------------------------------------------
begin_dt: datetime = datetime(2024, 3, 1)   # ← change as you like
end_dt:   datetime = datetime(2024, 3, 31)
resi_minors       = ["MG48", "MG50", "MG52", "MG55", "MG60"]
secondary_codes   = ["DDSB", "OPA"]
cml_minors        = ["CM06", "CM30", "CM52"]
disb_codes_cml    = ["PDSB", "SWPI"]
receipt_codes_cml = ["PRCT", "SWPR"]
# ------------------------------------------------------------------
# --- 2.  Helpers ---------------------------------------------------
# ------------------------------------------------------------------
def sql_list(py_list):
    """Turn ['A','B'] →  'A','B'  for SQL IN (...)"""
    return ", ".join(f"'{x}'" for x in py_list)
def to_date_literal(dt: datetime) -> str:
    """
    Return an Oracle-friendly TO_DATE literal.
      2024-03-15  →  TO_DATE('2024-03-15','YYYY-MM-DD')
    If the value has a time component, include it:
      2024-03-15 14:07:00  →  TO_DATE('2024-03-15 14:07:00','YYYY-MM-DD HH24:MI:SS')
    """
    fmt_date = "%Y-%m-%d %H:%M:%S" if isinstance(dt, datetime) and dt.time() != datetime.min.time() else "%Y-%m-%d"
    date_str = dt.strftime(fmt_date)
    mask     = "YYYY-MM-DD HH24:MI:SS" if " " in date_str else "YYYY-MM-DD"
    return f"TO_DATE('{date_str}','{mask}')"
# Pre-formatted pieces
resi_minors_sql        = sql_list(resi_minors)
secondary_codes_sql    = sql_list(secondary_codes)
cml_minors_sql         = sql_list(cml_minors)
disb_codes_cml_sql     = sql_list(disb_codes_cml)
receipt_codes_cml_sql  = sql_list(receipt_codes_cml)
start_date = "2025-03-01 00:00:00"
end_date = "2025-03-31 00:00:00"
# ------------------------------------------------------------------
# --- 3.  Compose the SQL ------------------------------------------
# ------------------------------------------------------------------

def fetch_from_database():
    """
    Gets data from COCC
    """
    rtxn = text(f"""
    SELECT
        a.RTXNNBR,
        a.RTXNTYPCD,
        a.ACCTNBR,
        a.TRANAMT
    FROM
        COCCDM.WH_RTXN a
    WHERE
        a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')
    """)

    queries = [
        {'key':'rtxn', 'sql':rtxn, 'engine':2},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [2]:
data = fetch_from_database()

In [3]:
rtxn = data['rtxn'].copy()

In [4]:
rtxn

,rtxnnbr,rtxntypcd,acctnbr,tranamt
0,8978,PWTH,26259192,-12.1
1,8977,DWTH,26259192,-32.95
2,8980,PWTH,26259192,-5.55
3,8972,PWTH,26259192,-1.89
4,8983,PWTH,26259192,-7
...,...,...,...,...
1035599,14090,XWTH,150293307,-32.63
1035600,7257,XWTH,910303,-32.39
1035601,8109,XWTH,150222322,-32.68
1035602,14091,XWTH,150293307,-32.68


In [5]:
def fetch_from_database():
    """
    Gets data from COCC
    """
    acctcommon = text(f"""
    SELECT
        a.ACCTNBR,
        a.CURRMIACCTTYPCD
    FROM
        OSIBANK.WH_ACCTCOMMON a
    """)

    queries = [
        {'key':'acctcommon', 'sql':acctcommon, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [6]:
data = fetch_from_database()

In [7]:
ac = data['acctcommon'].copy()

In [8]:
import cdutils.input_cleansing # type: ignore

rtxn_schema = {
    'acctnbr': str
}

rtxn = cdutils.input_cleansing.enforce_schema(rtxn, rtxn_schema)

ac_schema = {
    'acctnbr': str
}
ac = cdutils.input_cleansing.enforce_schema(ac, ac_schema)


In [9]:
rtxn = pd.merge(rtxn, ac, how='left', on='acctnbr')

In [10]:
rtxn

,rtxnnbr,rtxntypcd,acctnbr,tranamt,currmiaccttypcd
0,8978,PWTH,26259192,-12.1,CK04
1,8977,DWTH,26259192,-32.95,CK04
2,8980,PWTH,26259192,-5.55,CK04
3,8972,PWTH,26259192,-1.89,CK04
4,8983,PWTH,26259192,-7,CK04
...,...,...,...,...,...
1035599,14090,XWTH,150293307,-32.63,CK25
1035600,7257,XWTH,910303,-32.39,CK01
1035601,8109,XWTH,150222322,-32.68,CK05
1035602,14091,XWTH,150293307,-32.68,CK25


In [ ]:
resi_minors       = ["MG48", "MG50", "MG52", "MG55", "MG60"]
resi_codes        = ["PDSB","CWTH","CKUS","XDSB"]
secondary_codes   = ["PDSB", "OPA"]
cml_minors        = ["CM06", "CM30", "CM52"]
disb_codes_cml    = ["PDSB", "SWPI"]
receipt_codes_cml = ["PRCT", "SWPR"]

In [13]:
import numpy as np

In [ ]:
rtxn['adv_calc'] = abs(np.where((rtxn['currmiaccttypcd'].isin(resi_minors)) and (rtxn['tranamt'] < 0) and (rtxn['trantypcd'].isin(resi_codes)), rtxn['tranamt'],
                            (np.where(rtxn['trantypcd'].isin(secondary_codes) and (~(rtxn['currmiaccttypcd']).isin(cml_minors))), rtxn['tranamt'],
                            0)))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [1]:
"""
Balance Tracker - revised
Developed by CD
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.addition_fields
import src.balance_tracker_pipeline_v2
import src.cdutils.database.sliding_window
import src.excel_output
import src.monthly_delta
from src._version import __version__
import cdutils.distribution # type: ignore

# def main():
# Fetch Data from COCC
data_prior, data_current = src.cdutils.database.sliding_window.fetch_data()

_, data_prior_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)

In [4]:
df = full_data_current.copy()

In [5]:
# def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
"""
Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

Args:
    df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

Returns:
    df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
"""
# Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
df = df.copy()

# Adjustment for the Consumer loans: WSJ Prime + 1
df['modified_noteintrate'] = np.where(
    (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
    df['noteintrate']))))))))))))))
)


NameError: name 'np' is not defined

In [ ]:

# Attach dealer split rate and subtract this from noteint rate
df = cdutils.dealer_split.append_dealersplit(df)
print(df.info())
df['modified_noteintrate'] = np.where(df['Category'] == 'Indirect', df['modified_noteintrate'] - df['SPLT'], df['modified_noteintrate'])



# Create total yield
total_yield = weighted_avg_rate(df, title='Total Loan Yield')

# Create new loan yield
datetime_cols = ['effdate','origdate']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col])

new_loan_df = df[
    (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
].copy()

# # Attach transaction amt

# # Create advances column
# resi_minor_
# new_loan_df['Advances'] = np.where(
#     ()
# )

new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield')

df['Unadvanced'] = (df['availbalamt']) / 1000

# Unadvanced
unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

# Merge
merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

return merged_df


In [3]:
additional_fields = src.addition_fields.yield_and_unadvanced_creation(full_data_current)


Column 'acctuserfieldvalue' not found. Creating it with default None values.


TypeError: unsupported operand type(s) for -: 'float' and 'str'

In [ ]:


TEMPLATE_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
# OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD_test.xlsx')

src.excel_output.update_excel_template(TEMPLATE_PATH, monthly_delta, additional_fields, OUTPUT_PATH)

# Distribution 
recipients = [
    "chad.doorley@bcsbmail.com"
    # "Timothy.Chaves@bcsbmail.com",
    # "John.Silva@bcsbmail.com",
    # "Dawn.Young@bcsbmail.com",
    # "Christopher.Alves@bcsbmail.com",
    # "donna.oliveira@bcsbmail.com",
    # "nancy.pimentel@bcsbmail.com",
    # "Hasan.Ali@bcsbmail.com",
    # "Michael.Patacao@bcsbmail.com",
    # "Jeffrey.Pagliuca@bcsbmail.com",
    # "Erin.Riendeau@bcsbmail.com",
    # "donna.pavao@bcsbmail.com"
]
bcc_recipients = [
    "chad.doorley@bcsbmail.com",
    "businessintelligence@bcsbmail.com"
]
subject = f"Balance Tracker YTD - Through April 2025" 
body = "Hi all, \n\nAttached is the Balance Tracker through the most recent month end. If you have any questions, please reach out to BusinessIntelligence@bcsbmail.com\n"
attachment_paths = [OUTPUT_PATH]

cdutils.distribution.email_out(
    recipients = recipients, 
    bcc_recipients = bcc_recipients, 
    subject = subject, 
    body = body, 
    attachment_paths = attachment_paths
    )



if __name__ == '__main__':
print(f"Starting {__version__}")
main()
print("Complete!")



In [6]:
import cdutils.deduplication # type: ignore
import cdutils.database.connect # type: ignore
import cdutils.input_cleansing # type: ignore
from sqlalchemy import text # type: ignore
import pandas as pd

def fetch_from_acctuserfield():
        """
        Gets data from COCC
        """
        wh_acctuserfields = text(f"""
        SELECT
            *
        FROM 
            OSIBANK.WH_ACCTUSERFIELDS a
        """)

        queries = [
            {'key':'wh_acctuserfields', 'sql':wh_acctuserfields, 'engine':1},
        ]

        data = cdutils.database.connect.retrieve_data(queries)
        return data


# def append_dealersplit(df: pd.DataFrame):
"""
Attach secondary lending officer to any dataframe
"""

data = fetch_from_acctuserfield()

wh_acctuserfields = data['wh_acctuserfields'].copy()


In [7]:
wh_acctuserfields

,acctnbr,acctuserfieldcd,acctuserfieldcddesc,acctuserfieldvalue,acctuserfieldvaluedesc,acctdatelastmaint,rundate,datelastmaint
0,29017971,REOD,ATM/POS Opt In Flag,Y,Opt In,2018-08-04 14:21:26,2025-04-23,2025-04-23 21:37:39
1,150581645,SBAA,SBA Approval Date,2021-03-15,None,2021-05-11 17:14:56,2025-04-23,2025-04-23 21:37:39
2,1505954995,SPLT,Dealer Split Rate,0.0000,None,2021-04-13 13:45:07,2025-04-23,2025-04-23 21:37:39
3,60009047,REOD,ATM/POS Opt In Flag,N,Opt Out,2018-08-04 14:21:27,2025-04-23,2025-04-23 21:37:39
4,28180186,WITH,Withholding Status,4,"Exempt, Confirmed",2018-08-04 14:21:26,2025-04-23,2025-04-23 21:37:39
...,...,...,...,...,...,...,...,...
497561,150328815,PLED,Pledged Collateral,FRBB,FRBB,2019-06-11 11:15:56,2025-04-23,2025-04-23 21:37:39
497562,150328906,DLR,Dealer Name,First Hyundai,None,2019-06-11 12:10:00,2025-04-23,2025-04-23 21:37:39
497563,150326570,DDOC,Digital Document,LN_BILLS,D3 Electronic Loan Bill,2019-07-24 17:55:11,2025-04-23,2025-04-23 21:37:39
497564,150285510,DDOC,Digital Document,LN_BILLS,D3 Electronic Loan Bill,2019-08-15 18:07:31,2025-04-23,2025-04-23 21:37:39


In [8]:
splt = wh_acctuserfields[wh_acctuserfields['acctuserfieldcd'] == 'SPLT'].copy()

splt = splt.sort_values(by='datelastmaint', ascending=False).copy()
splt = cdutils.deduplication.dedupe([{'df':splt, 'field':'acctnbr'}])


# Asserts
assert splt['acctnbr'].is_unique, "splt not unique on acctnbr"


splt = splt[['acctnbr','acctuserfieldvalue']].copy()

splt = splt.rename(columns={'acctuserfieldvalue':'SPLT'}).copy()



In [9]:
splt

,acctnbr,SPLT
2,1505954995,0.0000
333735,13412781,.018675
333642,150671800,0.0000
333651,150666950,0.0175
333652,150744392,0.0200
...,...,...
164425,150708786,0.0000
164443,151013407,0.0000
164557,151099879,0.0175
164569,151141125,0.0000


In [10]:
splt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39769 entries, 2 to 497565
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   acctnbr  39769 non-null  int64 
 1   SPLT     39768 non-null  object
dtypes: int64(1), object(1)
memory usage: 932.1+ KB


In [ ]:
splt['SPLT'] = pd.to_numeric(splt['SPLT'], errors="coerce").fillna(0.0)


In [12]:


schema_df = {
            'acctnbr': str,
        }

df = cdutils.input_cleansing.enforce_schema(df, schema_df)

schema_splt = {
            'acctnbr': str,
            'SPLT': float
        }

splt = cdutils.input_cleansing.enforce_schema(splt, schema_splt)

assert df['acctnbr'].is_unique, "acctnbr not unique in df"

final_df = pd.merge(df, splt, on='acctnbr', how='left')

# return final_df




In [13]:
final_df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,fdiccatdesc,origbal,datemat,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,Category,SPLT
0,2025-03-31,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065747,...,RE/Multifamily,65500000.0,2028-08-01,29000000.00,0.000000e+00,0.0,2.900000e+07,65500000.0,CRE,NaN
1,2025-03-31,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,NonOwnOcc/RE/Non-Farm/Non-Res,26000000.0,2029-06-05,26000000.00,0.000000e+00,0.0,2.600000e+07,26000000.0,CRE,NaN
2,2025-03-31,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.0,FIX,CML,CM07,ACT,0.065000,...,Other Constr,20000000.0,2028-03-07,0.00,2.000000e+07,0.0,2.000000e+07,20000000.0,CRE,NaN
3,2025-03-31,151038843,"POWER 250, LLC",CML ARM Construction,27500000.0,VAR,CML,CM08,ACT,0.070701,...,Other Constr,27500000.0,2044-04-22,2254909.06,1.724509e+07,0.0,1.950000e+07,27500000.0,CRE,NaN
4,2025-03-31,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.0,FIX,CML,CM40,ACT,0.049000,...,NonOwnOcc/RE/Non-Farm/Non-Res,18000000.0,2027-11-01,17496659.09,0.000000e+00,0.0,1.749666e+07,18000000.0,CRE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23201,2025-03-31,970141021,"ARNOLD, NORMAN S.",Equity Line of Credit,50000.0,VAR,MTG,MG60,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,50000.0,2032-05-21,0.00,0.000000e+00,0.0,0.000000e+00,50000.0,Residential,NaN
23202,2025-03-31,970149281,"HUDSON, VICTORIA A.",Equity Line of Credit,25000.0,VAR,MTG,MG60,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,25000.0,2033-02-21,0.00,0.000000e+00,0.0,0.000000e+00,25000.0,Residential,NaN
23203,2025-03-31,970158171,"NOISEUX, KENNETH E.",Special HELOC,100000.0,VAR,MTG,MG52,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,100000.0,2033-12-13,0.00,0.000000e+00,0.0,0.000000e+00,100000.0,Residential,NaN
23204,2025-03-31,970186001,"SHURTLEFF, GIDEON N.",Special HELOC,38925.0,VAR,MTG,MG52,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,38925.0,2035-11-16,0.00,0.000000e+00,0.0,0.000000e+00,38925.0,Residential,NaN


In [ ]:
final_df['SPLT'] = pd.to_numeric(final_df['SPLT'], errors="coerce").fillna(0.0)


In [16]:
final_df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,fdiccatdesc,origbal,datemat,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,Category,SPLT
0,2025-03-31,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065747,...,RE/Multifamily,65500000.0,2028-08-01,29000000.00,0.000000e+00,0.0,2.900000e+07,65500000.0,CRE,0.0
1,2025-03-31,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,NonOwnOcc/RE/Non-Farm/Non-Res,26000000.0,2029-06-05,26000000.00,0.000000e+00,0.0,2.600000e+07,26000000.0,CRE,0.0
2,2025-03-31,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.0,FIX,CML,CM07,ACT,0.065000,...,Other Constr,20000000.0,2028-03-07,0.00,2.000000e+07,0.0,2.000000e+07,20000000.0,CRE,0.0
3,2025-03-31,151038843,"POWER 250, LLC",CML ARM Construction,27500000.0,VAR,CML,CM08,ACT,0.070701,...,Other Constr,27500000.0,2044-04-22,2254909.06,1.724509e+07,0.0,1.950000e+07,27500000.0,CRE,0.0
4,2025-03-31,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.0,FIX,CML,CM40,ACT,0.049000,...,NonOwnOcc/RE/Non-Farm/Non-Res,18000000.0,2027-11-01,17496659.09,0.000000e+00,0.0,1.749666e+07,18000000.0,CRE,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23201,2025-03-31,970141021,"ARNOLD, NORMAN S.",Equity Line of Credit,50000.0,VAR,MTG,MG60,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,50000.0,2032-05-21,0.00,0.000000e+00,0.0,0.000000e+00,50000.0,Residential,0.0
23202,2025-03-31,970149281,"HUDSON, VICTORIA A.",Equity Line of Credit,25000.0,VAR,MTG,MG60,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,25000.0,2033-02-21,0.00,0.000000e+00,0.0,0.000000e+00,25000.0,Residential,0.0
23203,2025-03-31,970158171,"NOISEUX, KENNETH E.",Special HELOC,100000.0,VAR,MTG,MG52,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,100000.0,2033-12-13,0.00,0.000000e+00,0.0,0.000000e+00,100000.0,Residential,0.0
23204,2025-03-31,970186001,"SHURTLEFF, GIDEON N.",Special HELOC,38925.0,VAR,MTG,MG52,ACT,0.075000,...,RE/1-4 Res/Rev Open-End,38925.0,2035-11-16,0.00,0.000000e+00,0.0,0.000000e+00,38925.0,Residential,0.0


In [20]:
final_df[final_df['Category'].isnull()]

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,fdiccatdesc,origbal,datemat,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,Category,SPLT
1546,2025-03-31,151119130,FRUITCAKE LLC,SWAP Exposure Loans,0.0,FIX,CML,CM09,ACT,0.0,...,Commercial Leases,NaN,2029-10-25,0.0,363600.0,0.0,363600.0,363600.0,NaN,0.0


In [1]:
### 2025-05-09
"""
Balance Tracker - revised
Developed by CD
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.addition_fields
import src.balance_tracker_pipeline_v3
import src.excel_output
import src.fetch_data
import src.monthly_delta
from src._version import __version__
import cdutils.distribution # type: ignore

# def main():
# Fetch Data from COCC
data_prior, data_current = src.fetch_data.fetch_data()



In [2]:
_, data_prior_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)
additional_fields = src.addition_fields.yield_and_unadvanced_creation(full_data_current)


#     TEMPLATE_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
#     OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
#     # OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD_test.xlsx')

#     src.excel_output.update_excel_template(TEMPLATE_PATH, monthly_delta, additional_fields, OUTPUT_PATH)

#     # Distribution 
#     recipients = [
#         # "chad.doorley@bcsbmail.com"
#         "Timothy.Chaves@bcsbmail.com",
#         "John.Silva@bcsbmail.com",
#         "Dawn.Young@bcsbmail.com",
#         "Christopher.Alves@bcsbmail.com",
#         "donna.oliveira@bcsbmail.com",
#         "nancy.pimentel@bcsbmail.com",
#         "Hasan.Ali@bcsbmail.com",
#         "Michael.Patacao@bcsbmail.com",
#         "Jeffrey.Pagliuca@bcsbmail.com",
#         "Erin.Riendeau@bcsbmail.com",
#         "donna.pavao@bcsbmail.com"
#     ]
#     bcc_recipients = [
#         "chad.doorley@bcsbmail.com",
#         "businessintelligence@bcsbmail.com"
#     ]
#     subject = f"Balance Tracker YTD - Through April 2025" 
#     body = "Hi all, \n\nAttached is the Balance Tracker through the most recent month end. If you have any questions, please reach out to BusinessIntelligence@bcsbmail.com\n\n"
#     attachment_paths = [OUTPUT_PATH]

#     cdutils.distribution.email_out(
#         recipients = recipients, 
#         bcc_recipients = bcc_recipients, 
#         subject = subject, 
#         body = body, 
#         attachment_paths = attachment_paths
#         )



# if __name__ == '__main__':
#     print(f"Starting {__version__}")
#     main()
#     print("Complete!")



In [3]:
full_data_current

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,rcf,availbalamt,fdiccatdesc,origbal,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,Category
0,2025-04-30,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.00,VAR,CML,CM40,ACT,0.065690,...,MNTH,0.0,RE/Multifamily,65500000.0,29000000.00,0.000000e+00,0.0,2.900000e+07,65500000.00,CRE
1,2025-04-30,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.00,FIX,CML,CM40,ACT,0.063500,...,None,0.0,NonOwnOcc/RE/Non-Farm/Non-Res,26000000.0,26000000.00,0.000000e+00,0.0,2.600000e+07,26000000.00,CRE
2,2025-04-30,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.00,FIX,CML,CM07,ACT,0.065000,...,None,20000000.0,Other Constr,20000000.0,0.00,2.000000e+07,0.0,2.000000e+07,20000000.00,CRE
3,2025-04-30,151038843,"POWER 250, LLC",CML ARM Construction,27500000.00,VAR,CML,CM08,ACT,0.070734,...,5YR,24320000.0,Other Constr,27500000.0,2254909.06,1.724509e+07,0.0,1.950000e+07,27500000.00,CRE
4,2025-04-30,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.00,FIX,CML,CM40,ACT,0.049000,...,None,0.0,NonOwnOcc/RE/Non-Farm/Non-Res,18000000.0,17465571.22,0.000000e+00,0.0,1.746557e+07,18000000.00,CRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83267,2025-04-30,970227181,"SULLIVAN, SEAN",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.037500,...,MNTH,0.0,RE/1-4 Res/Rev Open-End,NaN,0.00,0.000000e+00,0.0,0.000000e+00,0.00,Residential
83268,2025-04-30,970227201,"BALLOU, MARK E.",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.080000,...,MNTH,0.0,RE/1-4 Res/Rev Open-End,NaN,0.00,0.000000e+00,0.0,0.000000e+00,0.00,Residential
83269,2025-04-30,970227311,"LETENDRE, KAREN MARIE",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.032500,...,MNTH,0.0,RE/1-4 Res/Rev Open-End,NaN,0.00,0.000000e+00,0.0,0.000000e+00,0.00,Residential
83270,2025-04-30,970227551,"CAMPBELL, SCOTT A.",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.032500,...,MNTH,0.0,RE/1-4 Res/Rev Open-End,NaN,0.00,0.000000e+00,0.0,0.000000e+00,0.00,Residential


In [4]:
monthly_delta

,Category,Delta
1,CRE,34893.26136
0,C&I,875.96465
3,HOA,2492.69099
5,Residential,6938.62173
2,Consumer,-138.12243
4,Indirect,-245.91641


In [5]:
additional_fields

,Category,New Loan Yield,Total Loan Yield,Unadvanced Funds
0,C&I,0.080249,0.069191,140497.54457
1,CRE,0.068821,0.056849,320332.53329
2,Consumer,0.089664,0.079298,510.90830
3,HOA,0.068449,0.065733,18924.77974
4,Indirect,0.059254,0.051867,0.00000
5,Residential,0.063540,0.042644,83146.15747


In [6]:
df = full_data_current.copy()

In [7]:
"""
Weighted Average Rate & Unadvanced Funds
"""
import cdutils.dealer_split # type: ignore

import pandas as pd # type: ignore
import numpy as np # type: ignore


def weighted_avg_rate(df: pd.DataFrame, title: str='Weighted Avg Rate', weight_col: str = 'Net Balance', value_col: str = 'modified_noteintrate') -> pd.DataFrame:
    """
    Create weighted average rate
    """
    df = df.copy()
    df['WeightedRate'] = df[weight_col] * df[value_col]
    grouped_df = df.groupby('Category').apply(
        lambda x: x['WeightedRate'].sum() / x[weight_col].sum(), include_groups=False
    ).reset_index(name=title).copy()
    return grouped_df

# def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

#     Args:
#         df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

#     Returns:
#         df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
#     """
# Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
df = df.copy()

# Adjustment for the Consumer loans: WSJ Prime + 1
df['modified_noteintrate'] = np.where(
    (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
    df['noteintrate'])))))))))))))
)

# Attach dealer split rate and subtract this from noteint rate
df = cdutils.dealer_split.append_dealersplit(df)
df['modified_noteintrate'] = np.where(df['Category'] == 'Indirect', df['modified_noteintrate'] - df['SPLT'], df['modified_noteintrate'])



# Create total yield
total_yield = weighted_avg_rate(df, title='Total Loan Yield')

# Create new loan yield
datetime_cols = ['effdate','origdate']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col])

new_loan_df = df[
    (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
].copy()

# # Attach transaction amt

# # Create advances column
# resi_minor_
# new_loan_df['Advances'] = np.where(
#     ()
# )




In [8]:
new_loan_df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,fdiccatdesc,origbal,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,Category,modified_noteintrate,SPLT
13,2025-04-30,151167189,DARLING DEVELOPMENT CORPORATION,CML ARM Construction,12833000.00,VAR,CML,CM08,ACT,0.070693,...,Other Constr,12833000.0,0.00,12833000.00,0.0,12833000.00,12833000.00,CRE,0.070693,0.0
56,2025-04-30,151174887,SREC MASHPEE LLC,Construction to Perm,6532000.00,VAR,CML,CM81,ACT,0.067500,...,RE/Multifamily,6532000.0,5082000.00,1450000.00,0.0,6532000.00,6532000.00,CRE,0.067500,0.0
83,2025-04-30,151165943,"162 OLD COLONY, LLC",Commercial Mortgages,4560000.00,VAR,CML,CM40,ACT,0.066500,...,OwnOcc/RE/Non-Farm/Non-Res,4560000.0,4560000.00,0.00,0.0,4560000.00,4560000.00,CRE,0.066500,0.0
99,2025-04-30,151172188,CAMBRIDGE COHOUSING CONDOMINIUM ASSOCIATION,Community Assoc. Draw to Term,3925000.00,VAR,CML,CM47,ACT,0.065700,...,Comm & Industrial/US,3925000.0,4883.25,3920116.75,0.0,3925000.00,3925000.00,HOA,0.065700,0.0
109,2025-04-30,151169010,"SLV WALPOLE, LLC",Commercial Mortgages,3623000.00,FIX,CML,CM40,ACT,0.072500,...,RE/Multifamily,3623000.0,3623000.00,0.00,0.0,3623000.00,3623000.00,CRE,0.072500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20966,2025-04-30,151162452,"GOMES, BENJAMIN R.",Unsecured Loans,4600.00,FIX,CNS,IL30,ACT,0.095000,...,Consumer/Other,4600.0,4224.19,0.00,0.0,4224.19,4600.00,Consumer,0.095000,0.0
21662,2025-04-30,151165373,"MIND FULL CENTER FOR WELLNESS, LLC",ACH Manager,3000.00,FIX,CML,CI07,ACT,0.000000,...,None,3000.0,0.00,3000.00,0.0,3000.00,3000.00,CRE,0.000000,0.0
22950,2025-04-30,151166181,CHERYL I. MONIZ DBA ARTHUR MONIZ GALLERY,Express Business ODP,1000.00,FIX,CML,CM34,ACT,0.180000,...,Comm & Industrial/US,1000.0,0.00,1000.00,0.0,1000.00,1000.00,C&I,0.180000,0.0
23415,2025-04-30,151163939,"AMIRAULT, DOREEN F.",Overdraft LOC,500.00,FIX,CNS,IL75,ACT,0.180000,...,Other Revolving Credit Plans,500.0,0.00,500.00,0.0,500.00,500.00,Consumer,0.180000,0.0


In [13]:
slice = new_loan_df[new_loan_df['currmiaccttypcd'] == "CM06"].copy()

In [14]:
slice

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,fdiccatdesc,origbal,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,Category,modified_noteintrate,SPLT
1108,2025-04-30,151175546,"COASTLINE ELECTRIC, INC",Borrowing Base Line of Credit,500000.0,VAR,CML,CM06,ACT,0.085,...,Comm & Industrial/US,500000.0,193.5,499806.5,0.0,500000.0,500000.0,C&I,0.085,0.0


In [41]:
# Here we need to create a new column that would be the new balance + advances

In [1]:
start_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-01 00:00:00')
end_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-%d 00:00:00')


NameError: name 'new_loan_df' is not defined

In [22]:
from sqlalchemy import text # type: ignore


In [35]:
def fetch_rtxn():
    """
    Gets data from COCC
    """
    rtxn = text(f"""
    SELECT
        a.RTXNNBR,
        a.RTXNTYPCD,
        a.ACCTNBR,
        a.TRANAMT
    FROM
        COCCDM.WH_RTXN a
    WHERE
        a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')
    """)

    acctcommon = text(f"""
    SELECT
        a.ACCTNBR,
        a.CURRMIACCTTYPCD
    FROM
        OSIBANK.WH_ACCTCOMMON a
    """)

    queries = [
        {'key':'rtxn', 'sql':rtxn, 'engine':2},
        {'key':'acctcommon', 'sql':acctcommon, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [40]:
rtxn_data_pack = fetch_rtxn()

In [ ]:
acctcommon = rtxn_data_pack['acctcommon'].copy()
rtxn = rtxn_data_pack['rtxn'].copy()


In [ ]:
rtxn_schema = {'tranamt': 'float', 'acctnbr':'str'}
rtxn = cdutils.input_cleansing.enforce_schema(rtxn, rtxn_schema)

acctcommon_schema = {'acctnbr':'str'}
acctcommon = cdutils.input_cleansing.enforce_schema(acctcommon, acctcommon_schema)


In [44]:
rtxn = pd.merge(rtxn, acctcommon, how='left', on='acctnbr')

In [ ]:
rtxn

In [47]:
"""
resi_minors       = ["MG48", "MG50", "MG52", "MG55", "MG60"]
resi_codes        = ["PDSB","CWTH","CKUS","XDSB"]
secondary_codes   = ["PDSB", "OPA"]
cml_minors        = ["CM06", "CM30", "CM52"]
disb_codes_cml    = ["PDSB", "SWPI"]
receipt_codes_cml = ["PRCT", "SWPR"]
"""
resi_minors = ("MG48", "MG50", "MG52", "MG55", "MG60")
resi_codes = ("PDSB","CWTH","CKUS","XDSB")
secondary_codes = ("PDSB", "OPA")
cml_minors = ("CM06", "CM30", "CM52")
disb_codes_cml = ("PDSB", "SWPI")
receipt_codes_cml = ("PRCT", "SWPR")


In [77]:
def calculate_net_advances(row):
    if (row['currmiaccttypcd'] in resi_minors) and (row['tranamt'] < 0) and (row['rtxntypcd'] in resi_codes):
        return row['tranamt']
    elif (row['rtxntypcd'] in secondary_codes) and (row['currmiaccttypcd'] not in cml_minors):
        return row['tranamt']
    else:
        return 0

In [78]:
rtxn['net advances'] = rtxn.apply(calculate_net_advances, axis=1)

In [79]:
# rtxn['net advances'] = abs(rtxn['net advances'])

In [80]:
rtxn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033846 entries, 0 to 1033845
Data columns (total 6 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   rtxnnbr          1033846 non-null  int64  
 1   rtxntypcd        1033846 non-null  object 
 2   acctnbr          1033846 non-null  object 
 3   tranamt          1033846 non-null  float64
 4   currmiaccttypcd  1020672 non-null  object 
 5   net advances     1033846 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 47.3+ MB


In [ ]:
minor_df = rtxn.groupby('currmiaccttypcd')['net advances'].sum().reset_index()
minor_df['net advances'] = abs(minor_df['net advances'])
minor_df = minor_df.sort_values(by='net advances', ascending=False)
minor_df

,currmiaccttypcd,net advances
77,CM08,11145934.66
112,CM81,5082000.00
90,CM47,2310929.40
172,MG55,1891066.07
170,MG52,1303514.31
...,...,...
191,SV06,0.00
192,SV07,0.00
193,SV08,0.00
194,SV10,0.00


In [ ]:
pdsb = rtxn[rtxn['rtxntypcd'] == 'PDSB'].copy()

In [32]:
pdsb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 334 entries, 31451 to 1018522
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rtxnnbr    334 non-null    int64  
 1   rtxntypcd  334 non-null    object 
 2   acctnbr    334 non-null    int64  
 3   tranamt    334 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 13.0+ KB


In [33]:
pdsb.describe()

,rtxnnbr,acctnbr,tranamt
count,334.000000,3.340000e+02,3.340000e+02
mean,2332.733533,1.694563e+10,-1.028576e+05
std,732.745312,8.880640e+10,5.339333e+05
min,2.000000,1.037050e+05,-5.082000e+06
25%,2502.000000,1.507285e+08,-5.778207e+04
50%,2522.000000,1.510630e+08,-1.000000e+04
75%,2574.750000,1.511523e+08,-1.000000e+03
max,3036.000000,6.001158e+11,5.082000e+06


In [ ]:

new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield')

df['Unadvanced'] = (df['availbalamt']) / 1000

# Unadvanced
unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

# Merge
merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

    # return merged_df


In [1]:
"""
Balance Tracker - revised
Developed by CD
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.addition_fields
import src.balance_tracker_pipeline_v3
import src.excel_output
import src.fetch_data
import src.monthly_delta
from src._version import __version__
import cdutils.distribution # type: ignore

# def main():
# Fetch Data from COCC
data_prior, data_current = src.fetch_data.fetch_data()

_, data_prior_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)
# additional_fields = src.addition_fields.yield_and_unadvanced_creation(full_data_current)

In [90]:
"""
Weighted Average Rate & Unadvanced Funds
"""
import cdutils.dealer_split # type: ignore
import src.fetch_data

import pandas as pd # type: ignore
import numpy as np # type: ignore


def weighted_avg_rate(df: pd.DataFrame, title: str='Weighted Avg Rate', weight_col: str = 'Net Balance', value_col: str = 'modified_noteintrate') -> pd.DataFrame:
    """
    Create weighted average rate
    """
    df = df.copy()
    df['WeightedRate'] = df[weight_col] * df[value_col]
    grouped_df = df.groupby('Category').apply(
        lambda x: x['WeightedRate'].sum() / x[weight_col].sum(), include_groups=False
    ).reset_index(name=title).copy()
    return grouped_df

# def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
"""
Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

Args:
    df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

Returns:
    df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
"""
# Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
df = full_data_current.copy()

In [91]:
# Attempting to make this 1 query. Takes more work than expected
# import datetime
# from typing import Dict

# from sqlalchemy import text # type: ignore

# import cdutils.database.connect # Type: ignore

# def fetch_rtxn(start_date, end_date):
#     """
#     Gets data from COCC
#     """
#     rtxn = text(f"""
#     SELECT
#         t.RTXNNBR,
#         t.RTXNTYPCD,
#         t.ACCTNBR,
#         t.TRANAMT,
#         a.CURRMIACCTTYPCD
#     FROM
#         COCCDM.WH_RTXN t
#     JOIN
#         COCCDM.WH_ACCTCOMMON a
#         ON t.ACCTNBR = a.ACCTNBR
#         AND t.RUNDATE = a.EFFDATE
#     WHERE
#         (a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')) AND
#         (a.CURRRTXNSTATCD = 'C')
#     """)

#     acctcommon = text(f"""
#     SELECT
#         a.ACCTNBR,
#         a.CURRMIACCTTYPCD
#     FROM
#         COCCDM.WH_ACCTCOMMON_ME a
#     """)

#     queries = [
#         {'key':'rtxn', 'sql':rtxn, 'engine':2},
#         {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
#     ]

#     data = cdutils.database.connect.retrieve_data(queries)
#     return data

In [71]:

import datetime
from typing import Dict

from sqlalchemy import text # type: ignore

import cdutils.database.connect # Type: ignore

def fetch_rtxn(start_date, end_date):
    """
    Gets data from COCC
    """
    rtxn = text(f"""
    SELECT
        a.RTXNNBR,
        a.RTXNTYPCD,
        a.ACCTNBR,
        a.TRANAMT
    FROM
        COCCDM.WH_RTXN a
    WHERE
        (a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')) AND
        (a.CURRRTXNSTATCD = 'C')
    """)

    acctcommon = text(f"""
    SELECT
        a.ACCTNBR,
        a.CURRMIACCTTYPCD
    FROM
        COCCDM.WH_ACCTCOMMON_ME a
    """)

    queries = [
        {'key':'rtxn', 'sql':rtxn, 'engine':2},
        {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [92]:


# Adjustment for the Consumer loans: WSJ Prime + 1
df['modified_noteintrate'] = np.where(
    (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
    df['noteintrate'])))))))))))))
)

# Attach dealer split rate and subtract this from noteint rate
df = cdutils.dealer_split.append_dealersplit(df)
df['modified_noteintrate'] = np.where(df['Category'] == 'Indirect', df['modified_noteintrate'] - df['SPLT'], df['modified_noteintrate'])



# Create total yield
total_yield = weighted_avg_rate(df, title='Total Loan Yield')

# Create new loan yield
datetime_cols = ['effdate','origdate']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col])

new_loan_df = df[
    (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
].copy()

# Using the Transaction table
start_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-01 00:00:00')
end_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-%d 00:00:00')

# rtxn_data_pack = fetch_rtxn(start_date, end_date)

acctcommon = rtxn_data_pack['acctcommon'].copy()
rtxn = rtxn_data_pack['rtxn'].copy() 
rtxn_schema = {'tranamt': 'float', 'acctnbr':'str'}
rtxn = cdutils.input_cleansing.enforce_schema(rtxn, rtxn_schema)

acctcommon_schema = {'acctnbr':'str'}
acctcommon = cdutils.input_cleansing.enforce_schema(acctcommon, acctcommon_schema)
rtxn = pd.merge(rtxn, acctcommon, how='left', on='acctnbr')

# Calculting net advances
resi_minors = ("MG48", "MG50", "MG52", "MG55", "MG60")
resi_codes = ("PDSB","CWTH","CKUS","XDSB")
secondary_codes = ("PDSB", "OPA")
cml_minors = ("CM06", "CM30", "CM52")
disb_codes_cml = ("PDSB", "SWPI")
receipt_codes_cml = ("PRCT", "SWPR")

def calculate_advances(row):
    if (row['currmiaccttypcd'] in resi_minors) and (row['tranamt'] < 0) and (row['rtxntypcd'] in resi_codes):
        return row['tranamt']
    elif (row['rtxntypcd'] in secondary_codes) and (row['currmiaccttypcd'] not in cml_minors):
        return row['tranamt']
    else:
        return 0

rtxn['advances'] = rtxn.apply(calculate_advances, axis=1)

# Here we'll group by acctnbr and sum net advances
# Apply abs value to net advances

# Create a separate formula on transaction table for the 3 commercial minors that we need to net disb - reciepts
# or embed this into the second line of that formula above (couldn't get this correct based on the formula/codes Tom gave me)

# Then we merge it into new_load_df as an extra field and this is used in the weighted_avg_rate as the weight_col instead of Net Balance






In [93]:
# Need trancd formula built out: NDSB
# This will be a separate column that we add to advances
rtxn['new loan disb'] = np.where(rtxn['rtxntypcd'] == 'NDSB', rtxn['tranamt'], 0)

In [94]:
# CML
cml_minors = ("CM06", "CM30", "CM52")
# Group by acctnbr
disb_codes_cml = ("PDSB", "SWPI") # Increasing the loan
receipt_codes_cml = ("PRCT", "SWPR") # Paying down loan
# If positive, disb - receipt
# Else 0


In [97]:
rtxn['cml_disb'] = np.where((rtxn['rtxntypcd'].isin(disb_codes_cml)) & (rtxn['currmiaccttypcd'].isin(cml_minors)), rtxn['tranamt'], 0)
rtxn['cml_receipt'] = np.where((rtxn['rtxntypcd'].isin(receipt_codes_cml)) & (rtxn['currmiaccttypcd'].isin(cml_minors)), rtxn['tranamt'], 0)


In [98]:
acct_grouping = rtxn.groupby('acctnbr').agg({
    'advances':'sum',
    'new loan disb':'sum',
    'cml_disb':'sum',
    'cml_receipt':'sum'
}).reset_index()

In [99]:
acct_grouping

,acctnbr,advances,new loan disb,cml_disb,cml_receipt
0,1000001076,0.0,0.0,0.0,0.0
1,1000001175,0.0,0.0,0.0,0.0
2,1000001191,0.0,0.0,0.0,0.0
3,1000001207,0.0,0.0,0.0,0.0
4,1000001231,0.0,0.0,0.0,0.0
...,...,...,...,...,...
79035,99999162511967,0.0,0.0,0.0,0.0
79036,99999162511975,0.0,0.0,0.0,0.0
79037,99999162511977,0.0,0.0,0.0,0.0
79038,99999162511978,0.0,0.0,0.0,0.0


In [100]:
acct_grouping['advances'] = abs(acct_grouping['advances'])
acct_grouping['net cml advance'] = (acct_grouping['cml_disb'] + acct_grouping['cml_receipt']) * -1
acct_grouping['net cml advance'] = acct_grouping['net cml advance'].replace(-0.0,0.0)
acct_grouping['net cml advance'] = np.where(acct_grouping['net cml advance'] < 0, 0, acct_grouping['net cml advance'])
acct_grouping['advances'] = np.where(acct_grouping['net cml advance'] > 0, acct_grouping['net cml advance'], acct_grouping['advances'])
acct_grouping_final = acct_grouping[['acctnbr','advances','new loan disb']].copy()

# Merging
new_loan_df = pd.merge(df, acct_grouping_final, on='acctnbr', how='left')


In [107]:
new_loan_df['advances'] = new_loan_df['advances'].fillna(0)
new_loan_df['new loan disb'] = new_loan_df['new loan disb'].fillna(0)
new_loan_df['new_and_advanced'] = new_loan_df['advances'] + (new_loan_df['new loan disb'] * -1)

In [108]:
new_loan_df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,Category,modified_noteintrate,SPLT,advances,new loan disb,new_and_advanced
0,2025-04-30,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.00,VAR,CML,CM40,ACT,0.065690,...,0.000000e+00,0.0,2.900000e+07,65500000.00,CRE,0.065690,0.0,0.0,0.0,0.0
1,2025-04-30,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.00,FIX,CML,CM40,ACT,0.063500,...,0.000000e+00,0.0,2.600000e+07,26000000.00,CRE,0.063500,0.0,0.0,0.0,0.0
2,2025-04-30,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.00,FIX,CML,CM07,ACT,0.065000,...,2.000000e+07,0.0,2.000000e+07,20000000.00,CRE,0.065000,0.0,0.0,0.0,0.0
3,2025-04-30,151038843,"POWER 250, LLC",CML ARM Construction,27500000.00,VAR,CML,CM08,ACT,0.070734,...,1.724509e+07,0.0,1.950000e+07,27500000.00,CRE,0.070734,0.0,0.0,0.0,0.0
4,2025-04-30,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.00,FIX,CML,CM40,ACT,0.049000,...,0.000000e+00,0.0,1.746557e+07,18000000.00,CRE,0.049000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83267,2025-04-30,970227181,"SULLIVAN, SEAN",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.037500,...,0.000000e+00,0.0,0.000000e+00,0.00,Residential,0.037500,0.0,0.0,0.0,0.0
83268,2025-04-30,970227201,"BALLOU, MARK E.",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.080000,...,0.000000e+00,0.0,0.000000e+00,0.00,Residential,0.080000,0.0,0.0,0.0,0.0
83269,2025-04-30,970227311,"LETENDRE, KAREN MARIE",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.032500,...,0.000000e+00,0.0,0.000000e+00,0.00,Residential,0.032500,0.0,0.0,0.0,0.0
83270,2025-04-30,970227551,"CAMPBELL, SCOTT A.",Special HELOC,0.00,VAR,MTG,MG52,CLS,0.032500,...,0.000000e+00,0.0,0.000000e+00,0.00,Residential,0.032500,0.0,0.0,0.0,0.0


In [122]:
ytd_table_recon = new_loan_df.groupby('currmiaccttypcd')['new_and_advanced'].sum().sort_values(ascending=False).reset_index()

In [ ]:
ytd_table_recon6

,currmiaccttypcd,new_and_advanced
0,CM40,23373750.00
1,CM08,11145934.66
2,CM30,8581060.93
3,IL12,5896250.40
4,CM81,5082000.00
...,...,...
106,MG62,0.00
107,MG65,0.00
108,MG71,0.00
109,ML01,0.00


In [67]:
# minor_df = rtxn.groupby('currmiaccttypcd')['net advances'].sum().reset_index()
# minor_df['net advances'] = abs(minor_df['net advances'])
# minor_df = minor_df.sort_values(by='net advances', ascending=False)
# minor_df

In [111]:
new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield', weight_col='new_and_advanced')

df['Unadvanced'] = (df['availbalamt']) / 1000

# Unadvanced
unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

# Merge
merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

# return merged_df

In [112]:
merged_df

,Category,New Loan Yield,Total Loan Yield,Unadvanced Funds
0,C&I,0.077332,0.069191,140497.54457
1,CRE,0.071039,0.056849,320332.53329
2,Consumer,0.106493,0.079298,510.90830
3,HOA,0.066230,0.065733,18924.77974
4,Indirect,0.059249,0.051867,0.00000
5,Residential,0.065474,0.042644,83146.15747
